<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment3_LinearModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries that will be used 
import numpy as np
import tarfile
import glob
import re
import pandas as pd
#from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Loading the dataset

In [2]:
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz

--2021-10-04 21:55:25--  http://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4029756 (3.8M) [application/x-gzip]
Saving to: ‘scale_data.tar.gz.1’

scale_data.tar.gz.1 100%[===================>]   3.84M  17.8MB/s    in 0.2s    

2021-10-04 21:55:25 (17.8 MB/s) - ‘scale_data.tar.gz.1’ saved [4029756/4029756]



In [3]:
# Untar the dataset
my_tar = tarfile.open('/content/scale_data.tar.gz')
my_tar.extractall('/content/') 
my_tar.close()


In [20]:
id, review, rating, cl_3, cl_4 = [],[],[],[], []
for root in glob.glob ('/content/scaledata/*'):
  for path in glob.glob(root+'/*'): 
      fo = open(path)
      doc = fo.read()
      if 'id' in path:
        id.extend(doc.split('\n'))
      elif 'subj' in path:
        review.extend(doc.split('\n'))
      elif 'rating' in path:
        rating.extend(doc.split('\n'))
      elif '3class' in path:
        cl_3.extend(doc.split('\n'))
      else:
        cl_4.extend(doc.split('\n'))
      # print(path,'\n')
      # print(doc.split('\n')[:10])
print (len(id), len(review), len(rating), len(cl_3), len(cl_4))

data = pd.DataFrame(zip(id, review, rating, cl_3, cl_4), columns = ['id', 'review', 'rating', '3class', '4class'])
data = data[['review', '3class', '4class']].dropna()
data

5010 5010 5010 5010 5010


,review,3class,4class
0,"in my opinion , a movie reviewer's most import...",0,0
1,"you can watch this movie , that is based on a ...",0,0
2,"this is asking a lot to believe , and though i...",0,0
3,no heroes and no story are the main attributes...,0,0
4,"this is not an art movie , yet i saw it an art...",0,0
...,...,...,...
5005,nicolas roeg's mesmerizing 1971 film walkabout...,2,3
5006,the movie air force one should require a docto...,2,3
5007,""" well , jones , at least you haven't forgotte...",2,3
5008,in a time of bloated productions where special...,2,3


In [19]:
data['3class'].value_counts()

1    1915
2    1894
0    1197
        4
Name: 3class, dtype: int64

# Pre-processing the data

In [5]:
# Spliting the dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split (data['review'], data['3class'] , train_size = 0.8, random_state = 42, shuffle = True, stratify=data['3class'])
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (4008,) (4008,)
Shapes of X_test, y_test:  (1002,) (1002,)


In [6]:
# Vectorizing the documents
vectorizer = CountVectorizer(binary = True)
X_train = vectorizer.fit_transform(X_train.to_list())
X_test = vectorizer.transform(X_test.to_list())
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

Shapes of X_train, y_train:  (4008, 38596) (4008,)
Shapes of X_test, y_test:  (1002, 38596) (1002,)


# Linear models

### Default

In [9]:
def printing_eval_scores (y_true, y_pred, report=''):
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  precision = sklearn.metrics.precision_score(y_true, y_pred, average = 'weighted')
  recall = sklearn.metrics.recall_score(y_true, y_pred, average = 'weighted')
  f1 = sklearn.metrics.f1_score(y_true, y_pred, average = 'weighted')
  print('accuracy score: {:.3f}'.format(accuracy))
  print('precision score: {:.3f}'.format(precision))
  print('recall score: {:.3f}'.format(recall))
  print('F1 score: {:.3f}'.format(f1))
  if report is True:
    print(classification_report(y_true, y_pred))
  else:
    pass
  return accuracy, precision, recall, f1

In [22]:
def training (loss, penalty, max_iter): #penalty="l2", max_iter=100
  clf = SGDClassifier(loss=loss, penalty=penalty, max_iter=max_iter, shuffle=True).fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  printing_eval_scores (y_test, y_pred, report=True)

losses = ['log','hinge', 'perceptron' ]
penalties = ['l1','l2']
for penalty in penalties:
  for loss in losses:
    print('\n-------training linear model with loss = {}, penalty = {}'.format(loss,penalty ))
    training(loss = loss,penalty=penalty, max_iter=100 )



-------training linear model with loss = log, penalty = l1
accuracy score: 0.633
precision score: 0.671
recall score: 0.633
F1 score: 0.650
              precision    recall  f1-score   support

                   0.02      1.00      0.03         1
           0       0.69      0.64      0.66       239
           1       0.60      0.54      0.57       383
           2       0.74      0.72      0.73       379

    accuracy                           0.63      1002
   macro avg       0.51      0.73      0.50      1002
weighted avg       0.67      0.63      0.65      1002


-------training linear model with loss = hinge, penalty = l1
accuracy score: 0.646
precision score: 0.645
recall score: 0.646
F1 score: 0.645
              precision    recall  f1-score   support

                   1.00      1.00      1.00         1
           0       0.63      0.62      0.62       239
           1       0.58      0.57      0.58       383
           2       0.72      0.74      0.73       379

    accur

Penalty = L2, and loss = log (logistic regression model) yeild the best performance on F1 score (0.671)

### Searching the best parameter

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()